<a href="https://colab.research.google.com/github/CACMO/GeoAnalyzer_Cacmo/blob/main/Another_conversion_TESEC_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Cargar archivo CSV
archivo_real = "BZDT#115 Mainframe 815_(AGO.30.2024)DTA.CSV"  # Simula el archivo dado
# Read CSV, skipping 15 rows of metadata and using comma as separator
raw_df = pd.read_csv(archivo_real, skiprows=15, sep=',')

# Identify columns that represent individual tests (assuming they are purely numeric like '1', '2', '3')
test_columns = [col for col in raw_df.columns if str(col).isdigit()]

processed_data_list = []

# Extract attribute rows based on the 'Test' column
item_row = raw_df[raw_df['Test'] == 'Item'].iloc[0] if not raw_df[raw_df['Test'] == 'Item'].empty else pd.Series(dtype=object)
limit_units_row = raw_df[raw_df['Test'] == 'Limit Units'].iloc[0] if not raw_df[raw_df['Test'] == 'Limit Units'].empty else pd.Series(dtype=object)
bias1_value_row = raw_df[raw_df['Test'] == 'Bias 1 Value'].iloc[0] if not raw_df[raw_df['Test'] == 'Bias 1 Value'].empty else pd.Series(dtype=object)
bias1_label_row = raw_df[raw_df['Test'] == 'Bias 1 Unit'].iloc[0] if not raw_df[raw_df['Test'] == 'Bias 1 Unit'].empty else pd.Series(dtype=object)

# Check for 'Bin' row. If it exists, extract it. Otherwise, create a dummy one.
bin_row_exists = raw_df['Test'].astype(str).str.contains('Bin', na=False).any()
if bin_row_exists:
    bin_row = raw_df[raw_df['Test'].astype(str).str.contains('Bin', na=False)].iloc[0]
else:
    bin_row = pd.Series(index=raw_df.columns, dtype=object)
    for col in test_columns:
        bin_row[col] = np.nan # Fill with NaN for other columns

# Iterate through each test column (e.g., '1', '2', '3') to create structured data rows
for col_name in test_columns:
    test_dict = {
        "Test number": int(col_name),
        "Item": item_row.get(col_name, np.nan),
        "Bias 1 Label": bias1_label_row.get(col_name, np.nan),
        "Bias 1": pd.to_numeric(bias1_value_row.get(col_name, np.nan), errors='coerce'),
        "Units": limit_units_row.get(col_name, np.nan),
        "Bin": bin_row.get(col_name, np.nan)
    }
    processed_data_list.append(test_dict)

# Convert the list of dictionaries into the main DataFrame 'data'
data = pd.DataFrame(processed_data_list)

# Filtrar los datos excluyendo etiquetas "SAME" del campo 'Item'
data = data[~data['Item'].str.contains("SAME", na=False)]

# Configuración global - Crear el nombre para "Test name"
data['Test name'] = data['Item'] + "@" + data['Bias 1 Label'].astype(str) + data['Units']

# Definir límites (Hardcode aquí, deben procesarse según tabla real)
limites_pruebas = {
    "Bias 1": {"low_limit": 0.005, "high_limit": 0.020, "name": "Bias@A"},
    "Bias 2": {"low_limit": 0.01, "high_limit": 0.015, "name": "Bias@V"}
    # Agrega más límites y configuraciones si lo necesitas
}

# Generar resumen
resultados_pruebas = []
for index, row in data.iterrows():
    bias_value = row["Bias 1"]
    bias_unit = row["Units"]
    bias_name = row["Test name"]
    low_limit = limites_pruebas["Bias 1"]["low_limit"]
    high_limit = limites_pruebas["Bias 1"]["high_limit"]

    # Estadísticas generales
    if pd.api.types.is_number(bias_value):
        mean_value = bias_value
        stdev = 0 # Cannot calculate stdev for a single value
        p50 = bias_value
    elif pd.isnull(bias_value):
        mean_value = 0
        stdev = 0
        p50 = 0
    else: # Fallback for list-like bias_value, though not expected with current structure
        mean_value = np.mean(bias_value)
        stdev = np.std(bias_value)
        p50 = np.percentile(bias_value, 50)

    cp = (high_limit - low_limit) / (6 * stdev) if stdev != 0 else 0
    cpk = min((high_limit - mean_value) / (3 * stdev), (mean_value - low_limit) / (3 * stdev)) if stdev != 0 else 0

    # Resultados válidos y fallos
    if pd.api.types.is_number(bias_value):
        fail_count = 1 if (bias_value < low_limit) or (bias_value > high_limit) else 0
        result_count = 1
    elif pd.isnull(bias_value):
        fail_count = 0
        result_count = 0
    else: # Fallback for list-like bias_value
        fail_count = sum((bias_value < low_limit) | (bias_value > high_limit))
        result_count = len(bias_value)

    yield_percent = round(((result_count - fail_count) / result_count) * 100, 2) if result_count != 0 else 0

    resultados_pruebas.append({
        "Test number": row["Test number"],
        "Test name": bias_name,
        "Unit": row["Units"],
        "Soft Bin": row["Bin"],
        "Low limit": low_limit,
        "High limit": high_limit,
        "P50": round(p50, 4),
        "Mean": round(mean_value, 4),
        "Stdev": round(stdev, 4),
        "Cp": round(cp, 3),
        "Cpk": round(cpk, 3),
        "Result count": result_count,
        "Fail count": fail_count,
        "Yield (%)": yield_percent
    })

# Convertir resultados a DataFrame
tabla_final = pd.DataFrame(resultados_pruebas)

# Guardar tabla consolidada al CSV
tabla_final.to_csv("Resultados_Modificados_ZDT_PGlobal.csv", index=False)

# Mostrar tabla final simulada
print(tabla_final)

# Task
Modify the code in cell `zhn-EZXWWg4s` to add detailed comments explaining each section, variable, and logical step. Rewrite the statistical calculations to group the processed data by 'Test name', 'Unit', 'Soft Bin', 'Low limit', and 'High limit', then calculate the mean, standard deviation (Stdev), P50, Cp, Cpk, Result count, Fail count, and Yield (%) for each group using the collected 'Bias 1' values. Finally, provide a summary of the changes and updated results to address the issue of zero statistical values.

## Add Detailed Comments

### Subtask:
Add comprehensive comments to the existing code cell (`zhn-EZXWWg4s`) to explain each section, variable, and logical step in detail.


**Reasoning**:
The subtask requires adding comprehensive comments to the existing code cell `zhn-EZXWWg4s`. I will regenerate the entire code block with detailed comments explaining the purpose of the script, major sections, variable declarations, and logical steps, as instructed.



In [ ]:
# This script processes raw test data from a CSV file, extracts key attributes, calculates statistical metrics for each test,
# and consolidates the results into a final DataFrame for analysis and reporting.

# --- Import Libraries ---
# Import necessary libraries for data manipulation and numerical operations.
import pandas as pd
import numpy as np

# --- Load Data ---
# Define the input CSV file name and load the data into a Pandas DataFrame.
# The file is expected to contain metadata in the first 15 rows, which are skipped.
# The separator is explicitly set to a comma.
archivo_real = "BZDT#115 Mainframe 815_(AGO.30.2024)DTA.CSV"  # Specifies the name of the input CSV file containing raw test data.
raw_df = pd.read_csv(archivo_real, skiprows=15, sep=',') # DataFrame holding the raw data read from the CSV, skipping initial metadata rows.

# --- Identify Test Columns ---
# Identifies columns that represent individual test numbers by checking if their names are purely numeric.
# This ensures that only actual test data columns are considered for processing.
test_columns = [col for col in raw_df.columns if str(col).isdigit()]

# processed_data_list: An empty list to store dictionaries, each representing a processed test with its attributes and values.
processed_data_list = []

# --- Extract Attribute Rows ---
# These lines extract specific rows from the raw_df that contain meta-information for each test.
# They are identified by unique values in the 'Test' column (e.g., 'Item', 'Limit Units').
# .iloc[0] is used to get the first (and expected only) matching row.
# If a row is not found, an empty Series is created to prevent errors.
item_row = raw_df[raw_df['Test'] == 'Item'].iloc[0] if not raw_df[raw_df['Test'] == 'Item'].empty else pd.Series(dtype=object) # Series containing 'Item' labels for each test.
limit_units_row = raw_df[raw_df['Test'] == 'Limit Units'].iloc[0] if not raw_df[raw_df['Test'] == 'Limit Units'].empty else pd.Series(dtype=object) # Series containing 'Limit Units' for each test.
bias1_value_row = raw_df[raw_df['Test'] == 'Bias 1 Value'].iloc[0] if not raw_df[raw_df['Test'] == 'Bias 1 Value'].empty else pd.Series(dtype=object) # Series containing 'Bias 1 Value' for each test.
bias1_label_row = raw_df[raw_df['Test'] == 'Bias 1 Unit'].iloc[0] if not raw_df[raw_df['Test'] == 'Bias 1 Unit'].empty else pd.Series(dtype=object) # Series containing 'Bias 1 Unit' labels for each test.

# Check for 'Bin' row. If it exists, extract it. Otherwise, create a dummy one.
# The 'Bin' row indicates classification bins for tests, if available.
bin_row_exists = raw_df['Test'].astype(str).str.contains('Bin', na=False).any() # Boolean flag indicating if a 'Bin' row exists in the raw data.
if bin_row_exists:
    bin_row = raw_df[raw_df['Test'].astype(str).str.contains('Bin', na=False)].iloc[0] # Series containing 'Bin' information for each test, if present.
else:
    bin_row = pd.Series(index=raw_df.columns, dtype=object) # Dummy Series created if no 'Bin' row is found.
    for col in test_columns:
        bin_row[col] = np.nan # Fills 'Bin' values with NaN for each test column if no 'Bin' row exists.

# --- Process Test Data ---
# Iterates through each identified test column to consolidate its attributes into a dictionary.
# Each dictionary is then appended to `processed_data_list`.
for col_name in test_columns:
    # test_dict: A dictionary to hold attributes for the current test number.
    test_dict = {
        "Test number": int(col_name), # The numerical identifier for the test.
        "Item": item_row.get(col_name, np.nan), # The 'Item' (name) of the test.
        "Bias 1 Label": bias1_label_row.get(col_name, np.nan), # The label associated with 'Bias 1' (e.g., 'VSG').
        "Bias 1": pd.to_numeric(bias1_value_row.get(col_name, np.nan), errors='coerce'), # The numerical value of 'Bias 1', converted to numeric.
        "Units": limit_units_row.get(col_name, np.nan), # The units for the test limits.
        "Bin": bin_row.get(col_name, np.nan) # The bin classification for the test.
    }
    processed_data_list.append(test_dict)

# Convert the list of dictionaries into the main DataFrame 'data'.
data = pd.DataFrame(processed_data_list) # DataFrame created from the processed test dictionaries.

# --- Filter and Prepare Data ---
# Filters out rows where the 'Item' column contains the string "SAME".
# These rows typically represent duplicate or skipped tests and are not needed for final analysis.
data = data[~data['Item'].str.contains("SAME", na=False)] # Filters out tests marked as 'SAME'.

# Creates a new column 'Test name' by concatenating 'Item', 'Bias 1 Label', and 'Units'.
# This provides a more descriptive name for each test.
data['Test name'] = data['Item'] + "@" + data['Bias 1 Label'].astype(str) + data['Units'] # A descriptive name for each test, combining item, bias label, and units.

# --- Define Limits ---
# Hardcoded limits for different bias types. In a real scenario, these would likely be loaded from a configuration file or database.
# limites_pruebas: A dictionary defining the low and high limits for various bias tests, along with a descriptive name.
limites_pruebas = {
    "Bias 1": {"low_limit": 0.005, "high_limit": 0.020, "name": "Bias@A"},
    "Bias 2": {"low_limit": 0.01, "high_limit": 0.015, "name": "Bias@V"}
    # Add more limits and configurations if needed
}

# --- Calculate Statistics for Each Test ---
# resultados_pruebas: An empty list to store dictionaries, each containing calculated statistics for a test.
resultados_pruebas = []

# Iterates through each row (test) in the 'data' DataFrame to calculate statistical metrics.
for index, row in data.iterrows():
    bias_value = row["Bias 1"] # The measured 'Bias 1' value for the current test.
    bias_unit = row["Units"] # The unit of the 'Bias 1' value.
    bias_name = row["Test name"] # The descriptive name of the test.
    low_limit = limites_pruebas["Bias 1"]["low_limit"] # The lower limit for 'Bias 1' from the defined limits.
    high_limit = limites_pruebas["Bias 1"]["high_limit"] # The upper limit for 'Bias 1' from the defined limits.

    # --- General Statistics Calculation ---
    # Calculates mean, standard deviation, and P50 (median) for the bias value.
    # Handles cases where bias_value might be a single number, NaN, or (hypothetically) a list.
    if pd.api.types.is_number(bias_value): # Check if bias_value is a single numeric value.
        mean_value = bias_value # Mean is the value itself for a single point.
        stdev = 0 # Standard deviation is 0 for a single data point.
        p50 = bias_value # P50 (median) is the value itself for a single point.
    elif pd.isnull(bias_value): # Check if bias_value is NaN.
        mean_value = 0 # Default mean to 0 if value is missing.
        stdev = 0 # Default stdev to 0 if value is missing.
        p50 = 0 # Default P50 to 0 if value is missing.
    else: # Fallback for list-like bias_value, though not expected with current structure.
        mean_value = np.mean(bias_value)
        stdev = np.std(bias_value)
        p50 = np.percentile(bias_value, 50)

    # Cp (Process Capability Index): Measures if a process is capable of meeting specifications.
    # Calculated as (Upper Spec Limit - Lower Spec Limit) / (6 * Standard Deviation).
    # If stdev is 0, Cp is 0 to avoid division by zero.
    cp = (high_limit - low_limit) / (6 * stdev) if stdev != 0 else 0

    # Cpk (Process Capability Index, minimum of two sides): Measures how close the process mean is to the specification limits.
    # Calculated as the minimum of (USL - Mean) / (3 * Stdev) and (Mean - LSL) / (3 * Stdev).
    # If stdev is 0, Cpk is 0 to avoid division by zero.
    cpk = min((high_limit - mean_value) / (3 * stdev), (mean_value - low_limit) / (3 * stdev)) if stdev != 0 else 0

    # --- Valid Results and Failures Calculation ---
    # Determines the count of valid results and failures based on the defined limits.
    if pd.api.types.is_number(bias_value): # If bias_value is a single numeric value.
        fail_count = 1 if (bias_value < low_limit) or (bias_value > high_limit) else 0 # Counts as a failure if value is outside limits.
        result_count = 1 # There is 1 result for a single data point.
    elif pd.isnull(bias_value): # If bias_value is NaN.
        fail_count = 0 # No failures if no value.
        result_count = 0 # No results if no value.
    else: # Fallback for list-like bias_value.
        fail_count = sum((bias_value < low_limit) | (bias_value > high_limit))
        result_count = len(bias_value)

    # Yield (%): The percentage of results that fall within the specified limits.
    # Calculated as ((Total Results - Failures) / Total Results) * 100.
    # If result_count is 0, yield is 0 to avoid division by zero.
    yield_percent = round(((result_count - fail_count) / result_count) * 100, 2) if result_count != 0 else 0

    # Appends the calculated statistics for the current test to the list.
    resultados_pruebas.append({
        "Test number": row["Test number"],
        "Test name": bias_name,
        "Unit": row["Units"],
        "Soft Bin": row["Bin"],
        "Low limit": low_limit,
        "High limit": high_limit,
        "P50": round(p50, 4),
        "Mean": round(mean_value, 4),
        "Stdev": round(stdev, 4),
        "Cp": round(cp, 3),
        "Cpk": round(cpk, 3),
        "Result count": result_count,
        "Fail count": fail_count,
        "Yield (%)": yield_percent
    })

# --- Final Output ---
# Converts the list of statistical results into a final DataFrame.
tabla_final = pd.DataFrame(resultados_pruebas) # Final DataFrame containing all calculated statistics for each relevant test.

# Saves the final consolidated table to a CSV file without the DataFrame index.
tabla_final.to_csv("Resultados_Modificados_ZDT_PGlobal.csv", index=False)

# Prints the final DataFrame to the console for review.
print(tabla_final)
```


**Reasoning**:
The previous code block contained an extraneous ` (backtick) character at the end, which caused a `SyntaxError`. I need to remove this character to fix the syntax error.



In [ ]:
# This script processes raw test data from a CSV file, extracts key attributes, calculates statistical metrics for each test,
# and consolidates the results into a final DataFrame for analysis and reporting.

# --- Import Libraries ---
# Import necessary libraries for data manipulation and numerical operations.
import pandas as pd
import numpy as np

# --- Load Data ---
# Define the input CSV file name and load the data into a Pandas DataFrame.
# The file is expected to contain metadata in the first 15 rows, which are skipped.
# The separator is explicitly set to a comma.
archivo_real = "BZDT#115 Mainframe 815_(AGO.30.2024)DTA.CSV"  # Specifies the name of the input CSV file containing raw test data.
raw_df = pd.read_csv(archivo_real, skiprows=15, sep=',') # DataFrame holding the raw data read from the CSV, skipping initial metadata rows.

# --- Identify Test Columns ---
# Identifies columns that represent individual test numbers by checking if their names are purely numeric.
# This ensures that only actual test data columns are considered for processing.
test_columns = [col for col in raw_df.columns if str(col).isdigit()]

# processed_data_list: An empty list to store dictionaries, each representing a processed test with its attributes and values.
processed_data_list = []

# --- Extract Attribute Rows ---
# These lines extract specific rows from the raw_df that contain meta-information for each test.
# They are identified by unique values in the 'Test' column (e.g., 'Item', 'Limit Units').
# .iloc[0] is used to get the first (and expected only) matching row.
# If a row is not found, an empty Series is created to prevent errors.
item_row = raw_df[raw_df['Test'] == 'Item'].iloc[0] if not raw_df[raw_df['Test'] == 'Item'].empty else pd.Series(dtype=object) # Series containing 'Item' labels for each test.
limit_units_row = raw_df[raw_df['Test'] == 'Limit Units'].iloc[0] if not raw_df[raw_df['Test'] == 'Limit Units'].empty else pd.Series(dtype=object) # Series containing 'Limit Units' for each test.
bias1_value_row = raw_df[raw_df['Test'] == 'Bias 1 Value'].iloc[0] if not raw_df[raw_df['Test'] == 'Bias 1 Value'].empty else pd.Series(dtype=object) # Series containing 'Bias 1 Value' for each test.
bias1_label_row = raw_df[raw_df['Test'] == 'Bias 1 Unit'].iloc[0] if not raw_df[raw_df['Test'] == 'Bias 1 Unit'].empty else pd.Series(dtype=object) # Series containing 'Bias 1 Unit' labels for each test.

# Check for 'Bin' row. If it exists, extract it. Otherwise, create a dummy one.
# The 'Bin' row indicates classification bins for tests, if available.
bin_row_exists = raw_df['Test'].astype(str).str.contains('Bin', na=False).any() # Boolean flag indicating if a 'Bin' row exists in the raw data.
if bin_row_exists:
    bin_row = raw_df[raw_df['Test'].astype(str).str.contains('Bin', na=False)].iloc[0] # Series containing 'Bin' information for each test, if present.
else:
    bin_row = pd.Series(index=raw_df.columns, dtype=object) # Dummy Series created if no 'Bin' row is found.
    for col in test_columns:
        bin_row[col] = np.nan # Fills 'Bin' values with NaN for each test column if no 'Bin' row exists.

# --- Process Test Data ---
# Iterates through each identified test column to consolidate its attributes into a dictionary.
# Each dictionary is then appended to `processed_data_list`.
for col_name in test_columns:
    # test_dict: A dictionary to hold attributes for the current test number.
    test_dict = {
        "Test number": int(col_name), # The numerical identifier for the test.
        "Item": item_row.get(col_name, np.nan), # The 'Item' (name) of the test.
        "Bias 1 Label": bias1_label_row.get(col_name, np.nan), # The label associated with 'Bias 1' (e.g., 'VSG').
        "Bias 1": pd.to_numeric(bias1_value_row.get(col_name, np.nan), errors='coerce'), # The numerical value of 'Bias 1', converted to numeric.
        "Units": limit_units_row.get(col_name, np.nan), # The units for the test limits.
        "Bin": bin_row.get(col_name, np.nan) # The bin classification for the test.
    }
    processed_data_list.append(test_dict)

# Convert the list of dictionaries into the main DataFrame 'data'.
data = pd.DataFrame(processed_data_list) # DataFrame created from the processed test dictionaries.

# --- Filter and Prepare Data ---
# Filters out rows where the 'Item' column contains the string "SAME".
# These rows typically represent duplicate or skipped tests and are not needed for final analysis.
data = data[~data['Item'].str.contains("SAME", na=False)] # Filters out tests marked as 'SAME'.

# Creates a new column 'Test name' by concatenating 'Item', 'Bias 1 Label', and 'Units'.
# This provides a more descriptive name for each test.
data['Test name'] = data['Item'] + "@" + data['Bias 1 Label'].astype(str) + data['Units'] # A descriptive name for each test, combining item, bias label, and units.

# --- Define Limits ---
# Hardcoded limits for different bias types. In a real scenario, these would likely be loaded from a configuration file or database.
# limites_pruebas: A dictionary defining the low and high limits for various bias tests, along with a descriptive name.
limites_pruebas = {
    "Bias 1": {"low_limit": 0.005, "high_limit": 0.020, "name": "Bias@A"},
    "Bias 2": {"low_limit": 0.01, "high_limit": 0.015, "name": "Bias@V"}
    # Add more limits and configurations if needed
}

# --- Calculate Statistics for Each Test ---
# resultados_pruebas: An empty list to store dictionaries, each containing calculated statistics for a test.
resultados_pruebas = []

# Iterates through each row (test) in the 'data' DataFrame to calculate statistical metrics.
for index, row in data.iterrows():
    bias_value = row["Bias 1"] # The measured 'Bias 1' value for the current test.
    bias_unit = row["Units"] # The unit of the 'Bias 1' value.
    bias_name = row["Test name"] # The descriptive name of the test.
    low_limit = limites_pruebas["Bias 1"]["low_limit"] # The lower limit for 'Bias 1' from the defined limits.
    high_limit = limites_pruebas["Bias 1"]["high_limit"] # The upper limit for 'Bias 1' from the defined limits.

    # --- General Statistics Calculation ---
    # Calculates mean, standard deviation, and P50 (median) for the bias value.
    # Handles cases where bias_value might be a single number, NaN, or (hypothetically) a list.
    if pd.api.types.is_number(bias_value): # Check if bias_value is a single numeric value.
        mean_value = bias_value # Mean is the value itself for a single point.
        stdev = 0 # Standard deviation is 0 for a single data point.
        p50 = bias_value # P50 (median) is the value itself for a single point.
    elif pd.isnull(bias_value): # Check if bias_value is NaN.
        mean_value = 0 # Default mean to 0 if value is missing.
        stdev = 0 # Default stdev to 0 if value is missing.
        p50 = 0 # Default P50 to 0 if value is missing.
    else: # Fallback for list-like bias_value, though not expected with current structure.
        mean_value = np.mean(bias_value)
        stdev = np.std(bias_value)
        p50 = np.percentile(bias_value, 50)

    # Cp (Process Capability Index): Measures if a process is capable of meeting specifications.
    # Calculated as (Upper Spec Limit - Lower Spec Limit) / (6 * Standard Deviation).
    # If stdev is 0, Cp is 0 to avoid division by zero.
    cp = (high_limit - low_limit) / (6 * stdev) if stdev != 0 else 0

    # Cpk (Process Capability Index, minimum of two sides): Measures how close the process mean is to the specification limits.
    # Calculated as the minimum of (USL - Mean) / (3 * Stdev) and (Mean - LSL) / (3 * Stdev).
    # If stdev is 0, Cpk is 0 to avoid division by zero.
    cpk = min((high_limit - mean_value) / (3 * stdev), (mean_value - low_limit) / (3 * stdev)) if stdev != 0 else 0

    # --- Valid Results and Failures Calculation ---
    # Determines the count of valid results and failures based on the defined limits.
    if pd.api.types.is_number(bias_value): # If bias_value is a single numeric value.
        fail_count = 1 if (bias_value < low_limit) or (bias_value > high_limit) else 0 # Counts as a failure if value is outside limits.
        result_count = 1 # There is 1 result for a single data point.
    elif pd.isnull(bias_value): # If bias_value is NaN.
        fail_count = 0 # No failures if no value.
        result_count = 0 # No results if no value.
    else: # Fallback for list-like bias_value.
        fail_count = sum((bias_value < low_limit) | (bias_value > high_limit))
        result_count = len(bias_value)

    # Yield (%): The percentage of results that fall within the specified limits.
    # Calculated as ((Total Results - Failures) / Total Results) * 100.
    # If result_count is 0, yield is 0 to avoid division by zero.
    yield_percent = round(((result_count - fail_count) / result_count) * 100, 2) if result_count != 0 else 0

    # Appends the calculated statistics for the current test to the list.
    resultados_pruebas.append({
        "Test number": row["Test number"],
        "Test name": bias_name,
        "Unit": row["Units"],
        "Soft Bin": row["Bin"],
        "Low limit": low_limit,
        "High limit": high_limit,
        "P50": round(p50, 4),
        "Mean": round(mean_value, 4),
        "Stdev": round(stdev, 4),
        "Cp": round(cp, 3),
        "Cpk": round(cpk, 3),
        "Result count": result_count,
        "Fail count": fail_count,
        "Yield (%)": yield_percent
    })

# --- Final Output ---
# Converts the list of statistical results into a final DataFrame.
tabla_final = pd.DataFrame(resultados_pruebas) # Final DataFrame containing all calculated statistics for each relevant test.

# Saves the final consolidated table to a CSV file without the DataFrame index.
tabla_final.to_csv("Resultados_Modificados_ZDT_PGlobal.csv", index=False)

# Prints the final DataFrame to the console for review.
print(tabla_final)

## Rewrite Statistical Calculations

### Subtask:
Modify the code to group the processed data by 'Test name', 'Unit', 'Soft Bin', 'Low limit', and 'High limit', then calculate the mean, standard deviation (Stdev), P50, Cp, Cpk, Result count, Fail count, and Yield (%) for each group using the collected 'Bias 1' values.


**Reasoning**:
The subtask requires modifying the statistical calculation logic. I will define a new function `calculate_group_stats` to compute the required metrics for a group of 'Bias 1' values. Then, I will replace the existing row-wise iteration with a grouping operation using `groupby()` and `apply()` to aggregate statistics per defined group. The `tabla_final` DataFrame will be reconstructed from these grouped statistics. The `limites_pruebas` dictionary is already defined, and its values will be used.



In [ ]:
# This script processes raw test data from a CSV file, extracts key attributes, calculates statistical metrics for each test,
# and consolidates the results into a final DataFrame for analysis and reporting.

# --- Import Libraries ---
# Import necessary libraries for data manipulation and numerical operations.
import pandas as pd
import numpy as np

# --- Load Data ---
# Define the input CSV file name and load the data into a Pandas DataFrame.
# The file is expected to contain metadata in the first 15 rows, which are skipped.
# The separator is explicitly set to a comma.
archivo_real = "BZDT#115 Mainframe 815_(AGO.30.2024)DTA.CSV"  # Specifies the name of the input CSV file containing raw test data.
raw_df = pd.read_csv(archivo_real, skiprows=15, sep=',') # DataFrame holding the raw data read from the CSV, skipping initial metadata rows.

# --- Identify Test Columns ---
# Identifies columns that represent individual test numbers by checking if their names are purely numeric.
# This ensures that only actual test data columns are considered for processing.
test_columns = [col for col in raw_df.columns if str(col).isdigit()]

# processed_data_list: An empty list to store dictionaries, each representing a processed test with its attributes and values.
processed_data_list = []

# --- Extract Attribute Rows ---
# These lines extract specific rows from the raw_df that contain meta-information for each test.
# They are identified by unique values in the 'Test' column (e.g., 'Item', 'Limit Units').
# .iloc[0] is used to get the first (and expected only) matching row.
# If a row is not found, an empty Series is created to prevent errors.
item_row = raw_df[raw_df['Test'] == 'Item'].iloc[0] if not raw_df[raw_df['Test'] == 'Item'].empty else pd.Series(dtype=object) # Series containing 'Item' labels for each test.
limit_units_row = raw_df[raw_df['Test'] == 'Limit Units'].iloc[0] if not raw_df[raw_df['Test'] == 'Limit Units'].empty else pd.Series(dtype=object) # Series containing 'Limit Units' for each test.
bias1_value_row = raw_df[raw_df['Test'] == 'Bias 1 Value'].iloc[0] if not raw_df[raw_df['Test'] == 'Bias 1 Value'].empty else pd.Series(dtype=object) # Series containing 'Bias 1 Value' for each test.
bias1_label_row = raw_df[raw_df['Test'] == 'Bias 1 Unit'].iloc[0] if not raw_df[raw_df['Test'] == 'Bias 1 Unit'].empty else pd.Series(dtype=object) # Series containing 'Bias 1 Unit' labels for each test.

# Check for 'Bin' row. If it exists, extract it. Otherwise, create a dummy one.
# The 'Bin' row indicates classification bins for tests, if available.
bin_row_exists = raw_df['Test'].astype(str).str.contains('Bin', na=False).any() # Boolean flag indicating if a 'Bin' row exists in the raw data.
if bin_row_exists:
    bin_row = raw_df[raw_df['Test'].astype(str).str.contains('Bin', na=False)].iloc[0] # Series containing 'Bin' information for each test, if present.
else:
    bin_row = pd.Series(index=raw_df.columns, dtype=object) # Dummy Series created if no 'Bin' row is found.
    for col in test_columns:
        bin_row[col] = np.nan # Fills 'Bin' values with NaN for each test column if no 'Bin' row exists.

# --- Process Test Data ---
# Iterates through each identified test column to consolidate its attributes into a dictionary.
# Each dictionary is then appended to `processed_data_list`.
for col_name in test_columns:
    # test_dict: A dictionary to hold attributes for the current test number.
    test_dict = {
        "Test number": int(col_name), # The numerical identifier for the test.
        "Item": item_row.get(col_name, np.nan), # The 'Item' (name) of the test.
        "Bias 1 Label": bias1_label_row.get(col_name, np.nan), # The label associated with 'Bias 1' (e.g., 'VSG').
        "Bias 1": pd.to_numeric(bias1_value_row.get(col_name, np.nan), errors='coerce'), # The numerical value of 'Bias 1', converted to numeric.
        "Units": limit_units_row.get(col_name, np.nan), # The units for the test limits.
        "Bin": bin_row.get(col_name, np.nan) # The bin classification for the test.
    }
    processed_data_list.append(test_dict)

# Convert the list of dictionaries into the main DataFrame 'data'.
data = pd.DataFrame(processed_data_list) # DataFrame created from the processed test dictionaries.

# --- Filter and Prepare Data ---
# Filters out rows where the 'Item' column contains the string "SAME".
# These rows typically represent duplicate or skipped tests and are not needed for final analysis.
data = data[~data['Item'].str.contains("SAME", na=False)] # Filters out tests marked as 'SAME'.

# Creates a new column 'Test name' by concatenating 'Item', 'Bias 1 Label', and 'Units'.
# This provides a more descriptive name for each test.
data['Test name'] = data['Item'] + "@" + data['Bias 1 Label'].astype(str) + data['Units'] # A descriptive name for each test, combining item, bias label, and units.

# --- Define Limits ---
# Hardcoded limits for different bias types. In a real scenario, these would likely be loaded from a configuration file or database.
# limites_pruebas: A dictionary defining the low and high limits for various bias tests, along with a descriptive name.
limites_pruebas = {
    "Bias 1": {"low_limit": 0.005, "high_limit": 0.020, "name": "Bias@A"},
    "Bias 2": {"low_limit": 0.01, "high_limit": 0.015, "name": "Bias@V"}
    # Add more limits and configurations if needed
}

# --- Function to calculate group statistics ---
def calculate_group_stats(bias_values, low_limit, high_limit):
    """
    Calculates statistical metrics for a group of 'Bias 1' values.
    Args:
        bias_values (pd.Series): Series of 'Bias 1' values for a specific group.
        low_limit (float): The lower specification limit.
        high_limit (float): The upper specification limit.

    Returns:
        dict: A dictionary containing calculated statistics (Mean, Stdev, P50, Cp, Cpk, etc.).
    """
    # Convert to numeric, coercing errors to NaN
    numeric_bias_values = pd.to_numeric(bias_values, errors='coerce').dropna()

    result_count = len(numeric_bias_values)

    if result_count == 0:
        # No valid data points, return default values
        return {
            "P50": 0.0,
            "Mean": 0.0,
            "Stdev": 0.0,
            "Cp": 0.0,
            "Cpk": 0.0,
            "Result count": 0,
            "Fail count": 0,
            "Yield (%)": 0.0
        }

    mean_value = numeric_bias_values.mean()
    stdev = numeric_bias_values.std() if result_count > 1 else 0.0 # Stdev is 0 for a single data point
    p50 = numeric_bias_values.median()

    # Calculate Fail count
    fail_count = ((numeric_bias_values < low_limit) | (numeric_bias_values > high_limit)).sum()

    # Calculate Cp and Cpk, handling division by zero
    if stdev != 0:
        cp = (high_limit - low_limit) / (6 * stdev)
        cpk = min((high_limit - mean_value) / (3 * stdev), (mean_value - low_limit) / (3 * stdev))
    else:
        # If stdev is 0, process is perfectly centered or all values are identical.
        # If all values are within limits, Cp/Cpk can be considered infinite/very high.
        # If any value is outside limits, Cp/Cpk is 0 (or very bad).
        if (numeric_bias_values >= low_limit).all() and (numeric_bias_values <= high_limit).all():
            cp = np.inf # Or a very large number if np.inf causes issues downstream
            cpk = np.inf # Or a very large number
        else:
            cp = 0.0
            cpk = 0.0

    # Calculate Yield (%)
    yield_percent = ((result_count - fail_count) / result_count) * 100

    return {
        "P50": round(p50, 4),
        "Mean": round(mean_value, 4),
        "Stdev": round(stdev, 4),
        "Cp": round(cp, 3),
        "Cpk": round(cpk, 3),
        "Result count": result_count,
        "Fail count": fail_count,
        "Yield (%)": round(yield_percent, 2)
    }

# --- Apply Grouped Statistics ---
# Group the data by relevant attributes and apply the statistical calculation function.
# We will also keep the first 'Test number' from each group for identification purposes.

# Define grouping columns. 'Low limit' and 'High limit' are part of 'limites_pruebas' for 'Bias 1'.
group_cols = ['Test name', 'Unit', 'Soft Bin']

# Apply the function to each group
aggregated_stats = data.groupby(group_cols).apply(lambda x: calculate_group_stats(
    x['Bias 1'],
    limites_pruebas["Bias 1"]["low_limit"],
    limites_pruebas["Bias 1"]["high_limit"]
)).reset_index()

# The apply function returns a Series of dictionaries. We need to expand this into columns.
# Create a DataFrame from the aggregated statistics dictionaries
aggregated_stats_df = pd.json_normalize(aggregated_stats[0])

# Combine the grouping columns with the new statistics columns
tabla_final = pd.concat([aggregated_stats[group_cols], aggregated_stats_df], axis=1)

# Add Low limit and High limit columns back, as they are fixed per test type from limites_pruebas
tabla_final['Low limit'] = limites_pruebas["Bias 1"]["low_limit"]
tabla_final['High limit'] = limites_pruebas["Bias 1"]["high_limit"]

# Reorder columns to match the original output structure
tabla_final = tabla_final[[
    'Test name', 'Unit', 'Soft Bin', 'Low limit', 'High limit',
    'P50', 'Mean', 'Stdev', 'Cp', 'Cpk', 'Result count', 'Fail count', 'Yield (%)'
]]

# For 'Test number', we can either drop it as it might not be relevant for grouped stats,
# or take the minimum/maximum/first of the group. For now, it's not included in the grouped output directly.
# If a 'Test number' is required, an additional step to merge it from the original 'data' based on group_cols would be needed.
# As per the instruction, we're focusing on grouping by 'Test name', 'Unit', 'Soft Bin'.

# Guardar tabla consolidada al CSV
tabla_final.to_csv("Resultados_Modificados_ZDT_PGlobal.csv", index=False)

# Mostrar tabla final simulada
print(tabla_final)

**Reasoning**:
The previous code failed with a `KeyError: 'Unit'` because the column in the `data` DataFrame is named `Units` (plural) not `Unit` (singular). I need to correct the column name in the `group_cols` list and in the final column reordering to match the actual DataFrame column name.



In [ ]:
# This script processes raw test data from a CSV file, extracts key attributes, calculates statistical metrics for each test,
# and consolidates the results into a final DataFrame for analysis and reporting.

# --- Import Libraries ---
# Import necessary libraries for data manipulation and numerical operations.
import pandas as pd
import numpy as np

# --- Load Data ---
# Define the input CSV file name and load the data into a Pandas DataFrame.
# The file is expected to contain metadata in the first 15 rows, which are skipped.
# The separator is explicitly set to a comma.
archivo_real = "BZDT#115 Mainframe 815_(AGO.30.2024)DTA.CSV"  # Specifies the name of the input CSV file containing raw test data.
raw_df = pd.read_csv(archivo_real, skiprows=15, sep=',') # DataFrame holding the raw data read from the CSV, skipping initial metadata rows.

# --- Identify Test Columns ---
# Identifies columns that represent individual test numbers by checking if their names are purely numeric.
# This ensures that only actual test data columns are considered for processing.
test_columns = [col for col in raw_df.columns if str(col).isdigit()]

# processed_data_list: An empty list to store dictionaries, each representing a processed test with its attributes and values.
processed_data_list = []

# --- Extract Attribute Rows ---
# These lines extract specific rows from the raw_df that contain meta-information for each test.
# They are identified by unique values in the 'Test' column (e.g., 'Item', 'Limit Units').
# .iloc[0] is used to get the first (and expected only) matching row.
# If a row is not found, an empty Series is created to prevent errors.
item_row = raw_df[raw_df['Test'] == 'Item'].iloc[0] if not raw_df[raw_df['Test'] == 'Item'].empty else pd.Series(dtype=object) # Series containing 'Item' labels for each test.
limit_units_row = raw_df[raw_df['Test'] == 'Limit Units'].iloc[0] if not raw_df[raw_df['Test'] == 'Limit Units'].empty else pd.Series(dtype=object) # Series containing 'Limit Units' for each test.
bias1_value_row = raw_df[raw_df['Test'] == 'Bias 1 Value'].iloc[0] if not raw_df[raw_df['Test'] == 'Bias 1 Value'].empty else pd.Series(dtype=object) # Series containing 'Bias 1 Value' for each test.
bias1_label_row = raw_df[raw_df['Test'] == 'Bias 1 Unit'].iloc[0] if not raw_df[raw_df['Test'] == 'Bias 1 Unit'].empty else pd.Series(dtype=object) # Series containing 'Bias 1 Unit' labels for each test.

# Check for 'Bin' row. If it exists, extract it. Otherwise, create a dummy one.
# The 'Bin' row indicates classification bins for tests, if available.
bin_row_exists = raw_df['Test'].astype(str).str.contains('Bin', na=False).any() # Boolean flag indicating if a 'Bin' row exists in the raw data.
if bin_row_exists:
    bin_row = raw_df[raw_df['Test'].astype(str).str.contains('Bin', na=False)].iloc[0] # Series containing 'Bin' information for each test, if present.
else:
    bin_row = pd.Series(index=raw_df.columns, dtype=object) # Dummy Series created if no 'Bin' row is found.
    for col in test_columns:
        bin_row[col] = np.nan # Fills 'Bin' values with NaN for each test column if no 'Bin' row exists.

# --- Process Test Data ---
# Iterates through each identified test column to consolidate its attributes into a dictionary.
# Each dictionary is then appended to `processed_data_list`.
for col_name in test_columns:
    # test_dict: A dictionary to hold attributes for the current test number.
    test_dict = {
        "Test number": int(col_name), # The numerical identifier for the test.
        "Item": item_row.get(col_name, np.nan), # The 'Item' (name) of the test.
        "Bias 1 Label": bias1_label_row.get(col_name, np.nan), # The label associated with 'Bias 1' (e.g., 'VSG').
        "Bias 1": pd.to_numeric(bias1_value_row.get(col_name, np.nan), errors='coerce'), # The numerical value of 'Bias 1', converted to numeric.
        "Units": limit_units_row.get(col_name, np.nan), # The units for the test limits.
        "Bin": bin_row.get(col_name, np.nan) # The bin classification for the test.
    }
    processed_data_list.append(test_dict)

# Convert the list of dictionaries into the main DataFrame 'data'.
data = pd.DataFrame(processed_data_list) # DataFrame created from the processed test dictionaries.

# --- Filter and Prepare Data ---
# Filters out rows where the 'Item' column contains the string "SAME".
# These rows typically represent duplicate or skipped tests and are not needed for final analysis.
data = data[~data['Item'].str.contains("SAME", na=False)] # Filters out tests marked as 'SAME'.

# Creates a new column 'Test name' by concatenating 'Item', 'Bias 1 Label', and 'Units'.
# This provides a more descriptive name for each test.
data['Test name'] = data['Item'] + "@" + data['Bias 1 Label'].astype(str) + data['Units'] # A descriptive name for each test, combining item, bias label, and units.

# --- Define Limits ---
# Hardcoded limits for different bias types. In a real scenario, these would likely be loaded from a configuration file or database.
# limites_pruebas: A dictionary defining the low and high limits for various bias tests, along with a descriptive name.
limites_pruebas = {
    "Bias 1": {"low_limit": 0.005, "high_limit": 0.020, "name": "Bias@A"},
    "Bias 2": {"low_limit": 0.01, "high_limit": 0.015, "name": "Bias@V"}
    # Add more limits and configurations if needed
}

# --- Function to calculate group statistics ---
def calculate_group_stats(bias_values, low_limit, high_limit):
    """
    Calculates statistical metrics for a group of 'Bias 1' values.
    Args:
        bias_values (pd.Series): Series of 'Bias 1' values for a specific group.
        low_limit (float): The lower specification limit.
        high_limit (float): The upper specification limit.

    Returns:
        dict: A dictionary containing calculated statistics (Mean, Stdev, P50, Cp, Cpk, etc.).
    """
    # Convert to numeric, coercing errors to NaN
    numeric_bias_values = pd.to_numeric(bias_values, errors='coerce').dropna()

    result_count = len(numeric_bias_values)

    if result_count == 0:
        # No valid data points, return default values
        return {
            "P50": 0.0,
            "Mean": 0.0,
            "Stdev": 0.0,
            "Cp": 0.0,
            "Cpk": 0.0,
            "Result count": 0,
            "Fail count": 0,
            "Yield (%)": 0.0
        }

    mean_value = numeric_bias_values.mean()
    stdev = numeric_bias_values.std() if result_count > 1 else 0.0 # Stdev is 0 for a single data point
    p50 = numeric_bias_values.median()

    # Calculate Fail count
    fail_count = ((numeric_bias_values < low_limit) | (numeric_bias_values > high_limit)).sum()

    # Calculate Cp and Cpk, handling division by zero
    if stdev != 0:
        cp = (high_limit - low_limit) / (6 * stdev)
        cpk = min((high_limit - mean_value) / (3 * stdev), (mean_value - low_limit) / (3 * stdev))
    else:
        # If stdev is 0, process is perfectly centered or all values are identical.
        # If all values are within limits, Cp/Cpk can be considered infinite/very high.
        # If any value is outside limits, Cp/Cpk is 0 (or very bad).
        if (numeric_bias_values >= low_limit).all() and (numeric_bias_values <= high_limit).all():
            cp = np.inf # Or a very large number if np.inf causes issues downstream
            cpk = np.inf # Or a very large number
        else:
            cp = 0.0
            cpk = 0.0

    # Calculate Yield (%)
    yield_percent = ((result_count - fail_count) / result_count) * 100

    return {
        "P50": round(p50, 4),
        "Mean": round(mean_value, 4),
        "Stdev": round(stdev, 4),
        "Cp": round(cp, 3),
        "Cpk": round(cpk, 3),
        "Result count": result_count,
        "Fail count": fail_count,
        "Yield (%)": round(yield_percent, 2)
    }

# --- Apply Grouped Statistics ---
# Group the data by relevant attributes and apply the statistical calculation function.

# Define grouping columns. 'Low limit' and 'High limit' are part of 'limites_pruebas' for 'Bias 1'.
group_cols = ['Test name', 'Units', 'Soft Bin'] # Corrected 'Unit' to 'Units'

# Apply the function to each group
aggregated_stats = data.groupby(group_cols, dropna=False).apply(lambda x: calculate_group_stats(
    x['Bias 1'],
    limites_pruebas["Bias 1"]["low_limit"],
    limites_pruebas["Bias 1"]["high_limit"]
)).reset_index()

# The apply function returns a Series of dictionaries. We need to expand this into columns.
# Create a DataFrame from the aggregated statistics dictionaries
aggregated_stats_df = pd.json_normalize(aggregated_stats[0])

# Combine the grouping columns with the new statistics columns
tabla_final = pd.concat([aggregated_stats[group_cols], aggregated_stats_df], axis=1)

# Add Low limit and High limit columns back, as they are fixed per test type from limites_pruebas
tabla_final['Low limit'] = limites_pruebas["Bias 1"]["low_limit"]
tabla_final['High limit'] = limites_pruebas["Bias 1"]["high_limit"]

# Reorder columns to match the original output structure
tabla_final = tabla_final[[
    'Test name', 'Units', 'Soft Bin', 'Low limit', 'High limit',
    'P50', 'Mean', 'Stdev', 'Cp', 'Cpk', 'Result count', 'Fail count', 'Yield (%)'
]]

# For 'Test number', it is not directly included in this grouped output as per the instructions focusing on grouping by 'Test name', 'Units', 'Soft Bin'.

# Guardar tabla consolidada al CSV
tabla_final.to_csv("Resultados_Modificados_ZDT_PGlobal.csv", index=False)

# Mostrar tabla final simulada
print(tabla_final)
